This is a Notebook created with Insturctions for Import data feature for running import job inside a pipeline. 

It is important to follow the Pre-steps listed below carefully to ensure the proper working, else you might face issues and might have to do backtracking.
You can give an appropriate names to your resources by updating the name in the corresponding YAML files in the samples or by creating your own YAML using the sample and referencing it in the following when you are ready to create. These are just samples provided for you to get familiar with the required parameters that needs to be passed to create a new resource.


**1. Create test workspace with a compute instance 'cpu-instance'**

If you don't have an Azure subscription, sign up to try the [free or paid version of Azure Machine Learning](https://azure.microsoft.com/free/) today.
and follow the instructions to create a [AzureML workspace and compute instance](https://learn.microsoft.com/en-us/azure/machine-learning/quickstart-create-resources)

**2. Install SDKv2 private preview release**

In [ ]:
pip install azure-ai-ml==0.0.71493720 --extra-index-url https://azuremlsdktestpypi.azureedge.net/azureml-v2-cli-e2e-test/71493720/

In [9]:
from azure.identity import ManagedIdentityCredential
from azure.ai.ml import MLClient

subscription_id = '<your subscription here>'
resource_group = '<your rg here>'
workspace = '<your ws here>'
ml_client = MLClient(ManagedIdentityCredential(), subscription_id, resource_group, workspace)

**Check for existing Connections**

In [ ]:
#ml_client.connections.get(name=<connection_name>)

connection_list = ml_client.connections.list()

for conn in connection_list:
    print(conn)

**OR create a new connection by the follwing instructions in [create_ws_connections.ipynb](./create_ws_connection_sdk_v2.ipynb)**

**Import inside a Pipeline Job With AzSQL as an example**

In [15]:
from azure.ai.ml import load_component, dsl
from azure.ai.ml.entities import PipelineJob

def generate_dsl_pipeline():
    # 1. Load component funcs
    import_func = load_component("./job/import_component_test.yml")
    load_func = load_component("./job/load_component_test.yml")

    # 2. Construct pipeline
    @dsl.pipeline(compute="cpu-cluster", default_datastore="workspaceblobstore")
    def sample_pipeline():
        import_job = import_func(type="azuresqldb", connection="azureml:<your connection here>", query="<your query here>")
        load_job = load_func(imported_data=import_job.outputs.output)
        return { }

    pipeline = sample_pipeline()
    return pipeline

pipeline = generate_dsl_pipeline()
import_pipeline: PipelineJob = ml_client.jobs.create_or_update(job=pipeline)
